In [1]:
from sklearn import svm
import numpy as np
import pickle

with open('keyword_occurances.pkl','rb') as f:
    mydata = pickle.load(f)
    mydata = [map(int, x) for x in mydata]

target = [m[10] for m in mydata]
data = [j[:-1] for j in mydata]
    
inspire_X = np.asarray(data)
inspire_y = np.asarray(target)
inspire_y = inspire_y.ravel()

np.random.seed(0)
indices = np.random.permutation(len(inspire_X))

inspire_X_train = inspire_X[indices[:-160]]
inspire_y_train = inspire_y[indices[:-160]]
inspire_X_test  = inspire_X[indices[-160:]]
inspire_y_test  = inspire_y[indices[-160:]]

In [ ]:
clf = svm.SVC()
clf.fit(inspire_X_train, inspire_y_train)

In [ ]:
pred = clf.predict(inspire_X_test)
print pred
#print inspire_y_test

In [ ]:
error = np.mean( pred != inspire_y_test )
print (1-error)

In [ ]:
SUPPORTvec = clf.support_vectors_
SUPPORT    = clf.support_
nSUPPORT   = clf.n_support_

Multi-class fun

In [ ]:
clf1 = svm.SVC(decision_function_shape='ovo')
print clf1
clf1.fit(inspire_X_train, inspire_y_train)
pred1 = clf1.predict(inspire_X_test)
error1 = np.mean( pred1 != inspire_y_test )
print (1-error1)
print pred1

In [ ]:
print inspire_X_train.shape[1]
dec = clf1.decision_function(inspire_X_test)
#print dec

print inspire_X_test[0]
print dec[0]

we have 3 classes, A, B and C // 
dec gives a number for [AB, AC, BC] // 
close to zero means more unsure if its a or b, a or c, etc // 
positive number means its A rather than B, negative its B over A for AB // 
so for dec[0] we have [B,C,B] // 
theres 2 B's and only 1 C // 
therefore, B is what is predicted

unbalanced problemos // an attempt to compensate for the lack of class 0 objects.. dont think its worked

In [ ]:
clf2 = svm.SVC(decision_function_shape='ovo', class_weight={0: 10})
print clf2
clf2.fit(inspire_X_train, inspire_y_train)
pred2 = clf2.predict(inspire_X_test)
error2 = np.mean( pred2 != inspire_y_test )
print (1-error2)
print pred1
print inspire_y_test

scores

In [ ]:
svc = svm.SVC(decision_function_shape='ovo', )
#svc = svm.SVC(C=1, kernel='linear')
svc.fit(inspire_X_train, inspire_y_train).score(inspire_X_test, inspire_y_test)

In [ ]:
from sklearn.model_selection import KFold, cross_val_score
X = ["a", "a", "b", "c", "c", "c"]
k_fold = KFold(n_splits=3)
for train_indices, test_indices in k_fold.split(X):
     print('Train: %s | test: %s' % (train_indices, test_indices))

In [ ]:
[svc.fit(inspire_X[train], inspire_y[train]).score(inspire_X[test], inspire_y[test])
         for train, test in k_fold.split(inspire_X)]

In [ ]:
cross_val_score(svc, inspire_X, inspire_y, cv=k_fold, n_jobs=-1)

In [ ]:
cross_val_score(svc, inspire_X, inspire_y, cv=k_fold, scoring='precision_macro')

In [ ]:
X = inspire_X
y = inspire_y

#svc = svm.SVC(kernel='linear')
svc = svm.SVC(decision_function_shape='ovo')

C_s = np.logspace(-2, 1, 30)

scores = list()
scores_std = list()
for C in C_s:
    svc.C = C
    this_scores = cross_val_score(svc, X, y, n_jobs=1)
    scores.append(np.mean(this_scores))
    scores_std.append(np.std(this_scores))

# Do the plotting
import matplotlib.pyplot as plt
plt.figure(1, figsize=(4, 3))
plt.clf()
plt.semilogx(C_s, scores)
plt.semilogx(C_s, np.array(scores) + np.array(scores_std), 'b--')
plt.semilogx(C_s, np.array(scores) - np.array(scores_std), 'b--')
locs, labels = plt.yticks()
plt.yticks(locs, list(map(lambda x: "%g" % x, locs)))
plt.ylabel('CV score')
plt.xlabel('Parameter C')
plt.ylim(0.4, 0.55 )
plt.show()

finding a nice C! grid search!

In [ ]:
from sklearn.model_selection import GridSearchCV, cross_val_score
Cs = np.logspace(-2, 1, 20)
clf = GridSearchCV(estimator=svc, param_grid=dict(C=Cs), n_jobs=-1)

clf.fit(inspire_X_train, inspire_y_train) 

In [ ]:
print clf.best_score_ 
print clf.best_estimator_.C  